<a href="https://colab.research.google.com/github/gatherheart/DAppServer/blob/master/Baseline_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning, Spring 2020, Final Project

## Image Classification with Noisy Labels

1. Download ```final_project.pdf```, ```Kaggle & Colab Guide.pptx```, and ```utils.py``` from i-campus.
2. Go to [Kaggle competition page](https://www.naver.com), join Kaggle & competition, and download dataset.
3. Following guide slides, upload ```utils.py```.
4. Mount Google Drive.
5. Implement your own model and predict on test images.
6. Download and submit ```submission.csv``` to Kaggle.
7. Write a report on your project and submit on i-campus.

# INITIAL PACKAGES

In [0]:
# INITIAL PACKAGES
import os
import numpy as np
import pandas as pd
from time import time
from utils import run

## Mount Google Drive

Assmue you made ```final_project``` directory on the root,
and data files are there.

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [23]:
gdrive_root = '/content/gdrive/My Drive'
data_path = os.path.join(gdrive_root, 'HyunMuLab/Kaggle/ML_2020/final-project-introduction-to-ml-spring2020')
print(data_path)
os.listdir(data_path)

/content/gdrive/My Drive/HyunMuLab/Kaggle/ML_2020/final-project-introduction-to-ml-spring2020


['sample_submission.csv',
 'test_images.pkl',
 'train_data.pkl',
 'valid_data.pkl']

---

# SHOW YOUR WORK
From here, import packages you need as long as they are permitted. <br>
Fill ```train_and_predict``` function with your codes. <br>
If you want, you can implement your own classes or functions within "SHOW YOUR WOKR" block. <br>
The rest of work is ours.

In [0]:
# IMPORT PACKAGES YOU NEED
import numpy as np
from utils import run

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import copy

In [0]:
# YOUR OWN CLASSES OR FUNCTIONS

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.cnn_layers = nn.Sequential( # 8 CNN layers in total
            nn.Conv2d(3, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Conv2d(128, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(negative_slope=0.01),
            nn.MaxPool2d(2,2),
            nn.Dropout2d(p=0.25), # 3-layers
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Conv2d(256, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(negative_slope=0.01),
            nn.MaxPool2d(2,2),
            nn.Dropout2d(p=0.25),
            nn.Conv2d(512, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(negative_slope=0.01),
            nn.MaxPool2d(2,2)
        )

        self.linear = nn.Linear(4096, 14)
        
    def forward(self, x):
        x = self.cnn_layers(x)
        x = self.linear(x.view(x.shape[0], -1))
        return x

In [0]:
def train_and_predict(train_data, valid_data, test_images):
    """Train a model and return prediction on test images.

    Given train and valid data, build your model and optimize.
    Then, return predictions on test_images.

    You can import packages you want inside 'EDIT HERE' as long as they are permitted.
    (See document for the list of possible packages)

    arguments:
        train_data: tuple of (pandas.DataFrame, np.array).
        - 0: pandas.DataFrame with columns ['id', 'label']
          'id' contains unique id assigned to each image.
          'label' contains label (0 ~ # classes-1) corresponding to its image.
        - 1: train image in np.array of (# train data, # channel, height, width)

        valid_data: tuple of (pandas.DataFrame, np.array).
        - 0: pandas.DataFrame with columns ['id', 'label']
          'id' contains unique id assigned to each image.
          'label' contains label (0 ~ # classes-1) corresponding to its image.
        - 1: valid image in np.array of (# valid data, # channel, height, width)

        test_data: tuple of (pandas.DataFrame, np.array).
        - 0: pandas.DataFrame with columns ['id']
          'id' contains unique id assigned to each image.
        - 1: test image in np.array of (# test data, # channel, height, width)
    
    returns:
        pandas.DataFrame, predictions on test images with columns ['id', 'label'].
        'id' should contain unique id assigned to test images. 
        'label' should contain prediction on the test image correspond to its id

    """

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    LEARNING_RATE = 0.001
    BATCH_SIZE = 256
    TEST_BATCH_SIZE = 1024
    NUM_EPOCHS = 150
    PATIENCE = 10
    ENDURE = 0

    train_id_label, train_images = train_data['label'], train_data['image']
    valid_id_label, valid_images = valid_data['label'], valid_data['image']
    test_id, _test_images = test_images['id'], test_images['image']

    num_train = len(train_images)
    num_valid = len(valid_images)
    num_test = len(test_images)
    
    # Convert data into torch.Tensor
    x_train = torch.FloatTensor(train_images)
    y_train = train_id_label['label'].values
    y_train = torch.LongTensor(y_train)

    x_valid = torch.FloatTensor(valid_images)
    y_valid = valid_id_label['label'].values
    y_valid = torch.LongTensor(y_valid)

    x_test = torch.FloatTensor(_test_images)

    num_features = np.prod(x_train.shape[1:])
    num_classes = int(y_train.max()) + 1

    # Build torch dataset, dataloader
    train_dataset = TensorDataset(x_train, y_train)
    valid_dataset = TensorDataset(x_valid, y_valid)
    test_dataset = TensorDataset(x_test)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=False)

    # Build CNN model
    model = CNN().to(device)

    # Optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    loss_fn = nn.CrossEntropyLoss()


    # Train model
    mean_train_losses = []
    mean_valid_losses = []
    valid_acc_list = []
    best_acc = -1

    train_s = time()
    for epoch in range(1, NUM_EPOCHS + 1):
        epoch_s = time()
        model.train()
        
        train_losses = []
        valid_losses = []
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_losses.append(loss.item())
            
        model.eval()
        
        correct = 0
        total = 0
        # Disable gradient calculation for memory, computation efficiency
        with torch.no_grad():
            for i, (images, labels) in enumerate(valid_loader):
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = loss_fn(outputs, labels)
                
                valid_losses.append(loss.item())
                
                predicted = torch.argmax(outputs.data, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
                
        mean_train_losses.append(np.mean(train_losses))
        mean_valid_losses.append(np.mean(valid_losses))

        epoch_elapsed = time() - epoch_s
        
        valid_acc = correct / total
        valid_acc_list.append(valid_acc)
        print('epoch: {}, train loss: {:.4f}, valid loss: {:.4f}, valid acc: {:.4f}, elapsed: {:.4f}'\
            .format(epoch, np.mean(train_losses), np.mean(valid_losses), valid_acc, epoch_elapsed))
        
        if best_acc < valid_acc:
            print('Best Accuracy updated (%.4f => %.4f)' % (best_acc, valid_acc))
            best_acc = valid_acc
            best_epoch = epoch
            ENDURE = 0
            # Save best model
            torch.save(model.state_dict(), 'best_mlp.p')
        else:
            ENDURE += 1
            if ENDURE >= PATIENCE:
                print('Early stop triggered...!')
                break
    train_elapsed = time() - train_s

    print('Training Finished...!!')
    print('Time: %.4f' % train_elapsed)
    print('Best Valid acc : %.4f at epoch %d' % (best_acc, best_epoch))
    
    # Load best model
    model.load_state_dict(torch.load('best_mlp.p'))
    model.eval()

    # Make prediction on test data
    test_preds = []
    with torch.no_grad():
        for i, (images, ) in enumerate(test_loader):
            images = images.to(device)

            outputs = model(images)
            
            pred = outputs.argmax(1)

            if device == 'cuda':
                test_preds += pred.detach().cpu().numpy().tolist()
            else:
                test_preds += pred.detach().numpy().tolist()
    
    # Return prediction
    test_images['label'] = test_preds
    pred = test_images.loc[:, ['id', 'label']]

    return pred

---

# YOUR WORK IS DONE!
Do not touch any line below. <br>
```run``` function will grap your prediction and make ```submission.csv```. <br>
Take it and submit to Kaggle!

In [0]:
run(train_and_predict, data_path)

In [9]:
train_data

NameError: ignored

In [0]:
train_data = pd.read_pickle(os.path.join(data_path, 'train_data.pkl'))

In [0]:
_, label, image = train_data

In [12]:
import time

import torch
from matplotlib import pyplot as plt

from pykeops.torch import LazyTensor

use_cuda = torch.cuda.is_available()
dtype = 'float32' if use_cuda else 'float64'
torchtype = {'float32': torch.float32, 'float64': torch.float64}

ModuleNotFoundError: ignored

In [0]:
test_images = pd.read_pickle(os.path.join(data_path, 'test_images.pkl'))

In [0]:
_id, image = test_images

In [0]:
t, a = test_images

In [35]:
a

'image'

In [41]:
test_images['image']

27602    [[[[0.67843139 0.68627453 0.67843139 0.6941176...
5011     [[[[0.89803922 0.90196079 0.90588236 0.9058823...
2070     [[[[0.89411765 0.80000001 0.87058824 0.7529411...
11760    [[[[0.59215689 0.6156863  0.61960787 0.5647059...
7065     [[[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...
                               ...                        
26929    [[[[0.98823529 0.98823529 0.98823529 0.9882352...
27732    [[[[1.         1.         1.         1.       ...
12039    [[[[0.65490198 0.82352942 0.52549022 0.6196078...
2231     [[[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...
15612    [[[[0.52156866 0.52941179 0.56470591 0.4666666...
Name: image, Length: 10000, dtype: object

In [0]:
    valid_data = pd.read_pickle(os.path.join(data_path, 'valid_data.pkl'))


In [0]:
_, a, b = valid_data

In [44]:
b

'label'